In [1]:
from notebook_helpers import mprint

# Prompt Templates

A prompt-template is just a way to build different prompts to send to a chat model, based on pre-defined use-cases.

# DocSearchTemplate

A document search template is an object that searches a `DocumentIndex` based on a query, and inserts `n_docs` documents into the prompt, along with additional wording to the model asking it to use the information provided from the docs to answer the question.

In [2]:
from llm_workflow.indexes import ChromaDocumentIndex
from llm_workflow.prompt_templates import DocSearchTemplate

doc_index = ChromaDocumentIndex()
prompt_template = DocSearchTemplate(doc_index=doc_index, n_docs=1)

Here's the default prompt-template used by `DocSearchTemplate`:

In [3]:
mprint(prompt_template.template)


Answer the question at the end of the text as truthfully and accurately as possible, based on the following information provided.

Here is the information:

```
{{documents}}
```

Here is the question:

```
{{prompt}}
```


Let's add documents to our document index.

**If we pass a list of documents to `doc_index`, the `__call__` method will pass the list to the `add()` method. If we pass a string or Document to `dock_index`, the `__call__` method will pass the value to the `search()` method.**

In [4]:
from llm_workflow.base import Document

docs = [
    Document(
        content="The greatest basketball player of all time is Michael Jordan",
        metadata={'id': 1}
    ),
    Document(
        content="The greatest three point shooter of all time is Steph Curry.",
        metadata={'id': 0}
    ),
    Document(
        content="The greatest hockey player of all time is Wayne Gretzky.",
        metadata={'id': 2}
    ),
]
# passing list[Document] is equivalent of calling `doc_index.add(docs)`
doc_index(docs)

In [5]:
# passing a string (or Document) is equivalent of calling `doc_index.search(value)`
doc_index("Who is the greatest 3-point shooter of all time?", n_results=1)

[Document(content='The greatest three point shooter of all time is Steph Curry.', metadata={'id': 0, 'distance': 0.35710838437080383})]

Now, let's construct our prompt. The `DocSearchTemplate` object will retrieve the most relevant document (from the `ChromaDocumentIndex` object) based on the value we send it, and then inject that document into the prompt. Because we set `n_docs=1` above, it will only include one Document.

In [6]:
prompt = prompt_template("Who is the greatest 3-point shooter of all time?")
mprint(prompt)


Answer the question at the end of the text as truthfully and accurately as possible, based on the following information provided.

Here is the information:

```
The greatest three point shooter of all time is Steph Curry.
```

Here is the question:

```
Who is the greatest 3-point shooter of all time?
```


---